In [1]:
import json
import os
from tqdm import tqdm
from PIL import Image
from PIL import ImageDraw
import json
from multiprocessing import Pool

In [2]:
data_split = ['train','val']
curr_dir = os.getcwd()

path = os.path.join(curr_dir, 'data_subset')

In [3]:
fl = open('label2id.json')
label2id = json.load(fl)

fl.close()

In [4]:
fl = open('id2label.json')
id2label = json.load(fl)
fl.close()

In [5]:
def process_single_file(args):
    
    each_file,each_folder,combined_folder_path,new_path = args
    
    fl = open(combined_folder_path+'\\'+each_file)
    json_obj = json.load(fl)
    fl.close()

    json_data = json_obj
    img_height = json_data["imgHeight"]
    img_width = json_data["imgWidth"]

    # Create a blank image
    image = Image.new(mode="L",size= (img_width, img_height))
    draw = ImageDraw.Draw(image)

    # Draw polygons on the image
    for obj in json_data["objects"]:
        if obj["draw"]:
            label = obj["label"]
            correct_id = label2id[label]
            polygon = [(point[0], point[1]) for point in obj["polygon"]]
            draw.polygon(polygon,fill=correct_id)

    image.save(new_path+'\\'+each_folder+'\\'+each_file.replace("json","png"))
    
    print("\nprocessed image ",new_path+'\\'+each_folder+'\\'+each_file.replace("json","png"))

In [8]:


def preprocess_annotations(split):
    
    combined_path = path+'\gtFine\\'+split+'\\'
    
    new_path = path+'\label_processed'+'\\'+split
    
    folders = os.listdir(combined_path)
    
    try:
        os.mkdir(new_path)
    except Exception as e:
        print(e)
        print(new_path, " already exist")

    for each_folder in folders:
        
        combined_folder_path = combined_path+'\\'+each_folder

        files = os.listdir(combined_folder_path)
        
        try:
            os.mkdir(new_path+'\\'+each_folder)
        except:
            print(new_path+'\\'+each_folder," already exist")

        
        pool_args = []
        
        for each_file in files:
            
            pool_args.append([each_file,each_folder,combined_folder_path,new_path])
            
        pool = Pool(processes=20)
        pool.map(process_single_file,pool_args)
        
        
            
            

In [9]:
for each in data_split:
    preprocess_annotations(each)

[WinError 3] The system cannot find the path specified: 'd:\\Toronto MEng\\Winter23\\CSC2516 Neural Network and Deep learning\\Final Project\\CSC2516_Final_Project\\data_subset\\label_processed\\train'
d:\Toronto MEng\Winter23\CSC2516 Neural Network and Deep learning\Final Project\CSC2516_Final_Project\data_subset\label_processed\train  already exist
d:\Toronto MEng\Winter23\CSC2516 Neural Network and Deep learning\Final Project\CSC2516_Final_Project\data_subset\label_processed\train\3  already exist
